In [1]:
from google.colab import drive
drive.mount('./MyDrive')

Mounted at ./MyDrive


In [2]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.5 MB/s 
     |████████████████████████████████| 448 kB 50.2 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [3]:
import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import csv
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tqdm import tqdm

In [4]:
def save_csv(path, data):
  with open(path, 'w',newline='', encoding='utf-8-sig') as f:   
    # using csv.writer method from CSV package 
    write = csv.writer(f) 
    #write.writerow(fields) 
    write.writerows(data)

def load_csv(path):
  load_list = []
  f =  open(path, 'r', encoding='utf-8')
  reader = csv.reader(f) 
  for row in reader:
    load_list.append(row)
  f.close()
  return load_list

In [5]:
train = pd.read_csv('./MyDrive/MyDrive/DACON/bluehouse/data/train.csv', encoding = 'utf-8')
test = pd.read_csv('./MyDrive/MyDrive/DACON/bluehouse/data/test.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('./MyDrive/MyDrive/DACON/bluehouse/data/sample_submission.csv', encoding = 'utf-8')

In [6]:
train[train['data'].isnull()]

,index,category,data
2819,2819,0,NaN
3462,3462,1,NaN
20934,20934,2,NaN
22856,22856,2,NaN
26550,26550,2,NaN
28546,28546,2,NaN
31400,31400,1,NaN
36070,36070,0,NaN


In [ ]:
# 전처리

## 전처리
아래 과정을 실행한 결과를 train_okt.csv, test_okt.csv에 저장해 놨음  
load_csv로 불러다 쓰면 댐

In [8]:
train = train.dropna(how = 'any')
train['data'] = train['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test['data'] = test['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을']

In [9]:
okt=Okt()

In [ ]:
X_train = []
for sentence,i in zip(train['data'],tqdm(range(len(train['data'])))) :
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True)
    temp_X = [word for word in temp_X if not word in stopwords]
    X_train.append(temp_X)

 39%|███▉      | 15694/39992 [11:49<19:40, 20.59it/s]

In [ ]:
X_test = []
for sentence in test['data']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True)
    temp_X = [word for word in temp_X if not word in stopwords]
    X_test.append(temp_X)

In [ ]:
save_csv('./MyDrive/MyDrive/DACON/bluehouse/data/train_okt.csv', X_train)
save_csv('./MyDrive/MyDrive/DACON/bluehouse/data/test_okt.csv', X_test)

## 불러오기
위에서 okt로 전처리한 train, test 데이터 불러오기

In [18]:
X_train = load_csv('./MyDrive/MyDrive/DACON/bluehouse/data/train_okt.csv')
X_test = load_csv('./MyDrive/MyDrive/DACON/bluehouse/data/test_okt.csv')

In [19]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [20]:
vocab_size = 30000
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [21]:
max_len = 500
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

In [22]:
# 결측치는 일단 제외하고 훈련
y_train = to_categorical(train['category'][train['data'].isnull()==False])

In [23]:
# 모델 생성 및 훈련

In [24]:
model = Sequential()
model.add(Embedding(vocab_size, 120))
model.add(LSTM(120))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

history = model.fit(X_train, y_train, batch_size=128, epochs=15)

Epoch 1/15
313/313 [==============================] - 40s 108ms/step - loss: 0.5636 - acc: 0.7792
Epoch 2/15
313/313 [==============================] - 34s 107ms/step - loss: 0.4205 - acc: 0.8612
Epoch 3/15
313/313 [==============================] - 34s 108ms/step - loss: 0.3187 - acc: 0.8947
Epoch 4/15
313/313 [==============================] - 34s 108ms/step - loss: 0.2440 - acc: 0.9165
Epoch 5/15
313/313 [==============================] - 34s 108ms/step - loss: 0.2105 - acc: 0.9304
Epoch 6/15
313/313 [==============================] - 34s 108ms/step - loss: 0.1742 - acc: 0.9408
Epoch 7/15
313/313 [==============================] - 34s 107ms/step - loss: 0.1611 - acc: 0.9428
Epoch 8/15
313/313 [==============================] - 34s 107ms/step - loss: 0.1521 - acc: 0.9469
Epoch 9/15
313/313 [==============================] - 34s 107ms/step - loss: 0.1445 - acc: 0.9493
Epoch 10/15
313/313 [==============================] - 33s 107ms/step - loss: 0.1239 - acc: 0.9562
Epoch 11/15
313/313

In [ ]:
#6. 훈련된 모델로 예측, submission 파일을 생성합니다.

In [28]:
#6. 훈련된 모델로 예측, submission 파일을 생성합니다.

#y_pred = model.predict_classes(X_test)
y_pred=model.predict(X_test) 

In [29]:

#sample_submission['category'] = y_pred
sample_submission['category'] = y_pred.argmax(axis=1)
sample_submission.to_csv('./MyDrive/MyDrive/DACON/bluehouse/data/sub2.csv', encoding='utf-8', index = False)

In [30]:
y_pred

array([[2.5280431e-01, 2.8454966e-04, 7.4691111e-01],
       [3.6199966e-05, 6.0593538e-05, 9.9990320e-01],
       [3.0152982e-05, 9.9996245e-01, 7.3328647e-06],
       ...,
       [8.9193578e-05, 9.9986815e-01, 4.2654032e-05],
       [9.9966848e-01, 2.3671909e-04, 9.4903116e-05],
       [2.0545312e-04, 1.8472740e-04, 9.9960989e-01]], dtype=float32)

In [31]:
model.save('./MyDrive/MyDrive/DACON/bluehouse/model1')

INFO:tensorflow:Assets written to: ./MyDrive/MyDrive/DACON/bluehouse/model1/assets


INFO:tensorflow:Assets written to: ./MyDrive/MyDrive/DACON/bluehouse/model1/assets
